In [1]:
# https://github.com/azure/aml-real-time-ai


In [2]:
!az login
#!Connect-AzAccount


In [3]:
!Select-AzSubscription -Subscription "<YourSubscription>"

In [4]:
import os
import tensorflow as tf
import numpy as np

In [5]:
# Input images as a two-dimensional tensor containing an arbitrary number of images represented a strings
import azureml.contrib.brainwave.models.utils as utils
in_images = tf.placeholder(tf.string)
image_tensors = utils.preprocess_array(in_images)
print(image_tensors.shape)

(?, 224, 224, 3)

In [6]:
#Featurizer
#We use ResNet50 as a featurizer. In this step we initialize the model. This downloads a TensorFlow checkpoint of the quantized ResNet50.
from azureml.contrib.brainwave.models import QuantizedResnet50
model_path = os.path.expanduser('~/models')
model = QuantizedResnet50(model_path, is_frozen = True)
feature_tensor = model.import_graph_def(image_tensors)
print(model.version)
print(feature_tensor.name)
print(feature_tensor.shape)

1.1.2
resnet_v1_50/pool5_2:0
(?, 1, 1, 2048)

In [7]:
#Classifier
#The model we downloaded includes a classifier which takes the output of the ResNet50 and identifies an image. This classifier is trained on the ImageNet dataset. We are going to use this classifier for our service. The next #notebook shows how to train a classifier for a different data set. The input to the classifier is a tensor matching the output of our ResNet50 featurizer.

classifier_output = model.get_default_classifier(feature_tensor)

In [8]:
# Service Definition
#Now that we've definied the image preprocessing, featurizer, and classifier that we will execute on our service we can create a service definition. The service definition is a set of files generated from the model that #allow us to deploy to the FPGA service. The service definition consists of a pipeline. The pipeline is a series of stages that are executed in order. We support TensorFlow stages, Keras stages, and BrainWave stages. The #stages will be executed in order on the service, with the output of each stage input into the subsequent stage.
#To create a TensorFlow stage we specify a session containing the graph (in this case we are using the default graph) and the input and output tensors to this stage. We use this information to save the graph so that we can #execute it on the service.

from azureml.contrib.brainwave.pipeline import ModelDefinition, TensorflowStage, BrainWaveStage
save_path = os.path.expanduser('~/models/save')
model_def_path = os.path.join(save_path, 'model_def.zip')

model_def = ModelDefinition()
with tf.Session() as sess:
    model_def.pipeline.append(TensorflowStage(sess, in_images, image_tensors))
    model_def.pipeline.append(BrainWaveStage(sess, model))
    model_def.pipeline.append(TensorflowStage(sess, feature_tensor, classifier_output))
    model_def.save(model_def_path)
    print(model_def_path)

INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
INFO:tensorflow:Restoring parameters from /root/models/msfprn50/1.1.2/resnet50_bw
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
/root/models/save/model_def.zip

In [9]:
# Generate Workspace File for next step
#import azureml.core
#print(azureml.core.VERSION)

#from azureml.core import Workspace
#ws = Workspace.create(name='jbfpgamlws',
#                      subscription_id='82d7b191-5a8c-4cbf-a9f9-9aa5fb50feaa',	
#                      resource_group='jbfpga',
#                      create_resource_group=False,
#                      location='eastus2' 
#                     )
#ws.get_details()



In [10]:
#dbutils.fs.put("config.json", "{'subscription_id': '82d7b191-5a8c-4cbf-a9f9-9aa5fb50feaa', 'resource_group': 'jbfpga','workspace_name': 'jbfpgamlws'}")

In [11]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<YourSubscription ID>")
resource_group = os.getenv("RESOURCE_GROUP", default="<YourResourcegroup>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<YourWorkspace>")
workspace_region = os.getenv("WORKSPACE_REGION", default="<YourAzureDCLocation>")

In [12]:
#Access your workspace
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

Wrote the config file config.json to: /databricks/driver/aml_config/config.json
Workspace configuration succeeded. Skip the workspace creation steps below

In [13]:
#Deploy
#Time to create a service from the service definition. You need a Workspace in the East US 2 location. In the previous notebooks, you've created this Workspace. The code below will load that Workspace from a configuration #file.
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


Found the config file in: /databricks/driver/aml_config/config.json
jbfpgamlws
jbfpga
eastus2
82d7b191-5a8c-4cbf-a9f9-9aa5fb50feaa

In [14]:
#Upload the model to the workspace.
from azureml.core.model import Model

model_name = "resnet-50-rtai"
#model_def_path = "/databricks/driver/aml_config/"
#model_def_path = os.path.expanduser('~/')
save_path = os.path.expanduser('~/models/save')
model_def_path = os.path.join(save_path, 'model_def.zip')

registered_model = Model.register(ws, model_def_path, model_name)



Registering model resnet-50-rtai

In [15]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

from azureml.core.model import Model
model_name = "resnet-50-rtai"
save_path = os.path.expanduser('~/models/save')
model_def_path = os.path.join(save_path, 'model_def.zip')
registered_model = Model.register(ws, model_def_path, model_name)

from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.contrib.brainwave import BrainwaveWebservice, BrainwaveImage
service_name = "imagenet-infer"
service = None
try:
    service = Webservice(ws, service_name)
except WebserviceException:
    image_config = BrainwaveImage.image_configuration()
    deployment_config = BrainwaveWebservice.deploy_configuration()
    
    print(ws, service_name,[registered_model],image_config,deployment_config, sep = '\n')
    
    service = Webservice.deploy_from_model(ws, service_name, [registered_model], image_config, deployment_config)
    service.wait_for_deployment(True)

Found the config file in: /databricks/driver/aml_config/config.json
jbfpgamlws
jbfpga
eastus2
82d7b191-5a8c-4cbf-a9f9-9aa5fb50feaa
Registering model resnet-50-rtai
<azureml.core.workspace.Workspace object at 0x7f86fb54aa20>
imagenet-infer
[<azureml.core.model.Model object at 0x7f86fb54ac50>]
<azureml.contrib.brainwave.brainwave_image.BrainwaveImageConfiguration object at 0x7f8702486780>
<azureml.contrib.brainwave.brainwave_webservice.RealTimeAIWebserviceDeploymentConfiguration object at 0x7f87306d2c18>
Creating image
Image creation operation finished for image imagenet-infer:19, operation "Succeeded"
Creating service
Running....................................
SucceededFPGA service creation operation finished, operation "Succeeded"

In [16]:
#Client
#The service supports gRPC and the TensorFlow Serving "predict" API. We provide a client that can call the service to get predictions on aka.ms/rtai. You can also invoke the service like any other web service.
#
#To understand the results we need a mapping to the human readable imagenet classes
import requests
classes_entries = requests.get("https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt").text.splitlines()

from azure.storage.blob import BlockBlobService
block_blob_service = BlockBlobService(account_name='<YourStorageAcctName>', account_key='<YourStorageAcctKey>')
block_blob_service.get_blob_to_path('images', 'Snowleopard.jpg', 'TestImage.jpg')

#https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-consume-web-service

print(service.scoring_uri)
primary, secondary = service.get_keys()
print(primary)
print(secondary)

#service = Webservice(workspace=ws, name='imagenet-infer')
print(service.scoring_uri)

# We can now send an image to the service and get the predictions. Let's see if it can identify a snow leopard. 
results = service.run('TestImage.jpg')
# map results [class_id] => [confidence]
results = enumerate(results)
# sort results by confidence
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
# print top 5 results
for top in sorted_results[:5]:
    print(classes_entries[top[0]], 'confidence:', top[1])

168.62.38.171:80
Q20U5fgJh99gVVcjFDprM1FHvGuxuUSE
51sYQX2PFbjaXuyg8nyqBhZsFjrnqiGC
168.62.38.171:80
snow leopard, ounce, Panthera uncia confidence: 0.9519317
leopard, Panthera pardus confidence: 0.047500856
jaguar, panther, Panthera onca, Felis onca confidence: 0.00023210353
cheetah, chetah, Acinonyx jubatus confidence: 0.00017502057
lynx, catamount confidence: 7.8550125e-05